In [51]:
# -*- coding:utf-8 -*-
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
import cv2
import os

orignal =[]


In [52]:
#Read all test videos
test=[]
f= 'sample_images_RA'
Test_videos =[]
test_list = open('Labels/test.txt','r')
for line in test_list:
    name = line.split(" ")[0]
    x=line.split(" ")[-1]
    test.append(x[0])
    Test_videos.append(f+"/"+name)
    

In [53]:
for i in test:
    if i=='0':
        orignal.append('normal')
    else:
        orignal.append('abnormal')

In [54]:
#To load pre-trained model, trained on 5 classes 
#model = tf.keras.models.load_model('Trained Models/model_Q3D.h5',compile=False)
#model.summary()

#To load pre-trained weeights,
model = tf.keras.models.load_model('Trained Models/weights_model_Q3D.hdf5',compile=False)

In [55]:
#two classes labesl
labels0 ={0:'normal', 1:'abnormal'}
prediction = []

In [56]:
def add_salt_and_pepper_noise(frame, salt_prob=0.1, pepper_prob=1):
    noisy_frame = frame.copy()
    total_pixels = frame.size
    num_salt = int(total_pixels * salt_prob)
    num_pepper = int(total_pixels * pepper_prob)

    # Add Salt noise
    coords = [np.random.randint(0, i - 1, num_salt) for i in frame.shape]
    noisy_frame[coords[0], coords[1], :] = 255

    # Add Pepper noise
    coords = [np.random.randint(0, i - 1, num_pepper) for i in frame.shape]
    noisy_frame[coords[0], coords[1], :] = 0

    return noisy_frame


In [57]:
def median_filtering(image, kernel_size=3):
    return cv2.medianBlur(image, kernel_size)

In [ ]:
#Code for Confusion Matrix
for video in Test_videos:
    vid = []
    nor=0
    abnor=0
    path=video
    images = os.listdir(video)
    for img in images:
        img_path=path +"/"+img
        img2 = cv2.imread(img_path)
        img2 = add_salt_and_pepper_noise(img2, salt_prob=0.1, pepper_prob=1)
        img2 = median_filtering(img2)
        img2= cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        vid.append(cv2.resize(img2, (112, 112)))
    vid = np.array(vid, dtype=np.float32)
    length =len(vid)
    for i in range(0,len(vid),32):
        if abnor==3:
            break
        if i+32<=length:
            X = vid[i:i+32].transpose((1,2,0,3))
            output = model.predict_on_batch(np.array([X]))
            if(len(output)>0):
                str1=labels0[np.argmax(output)]
                if str1=='abnormal':
                    abnor=abnor+1
                else:
                    abnor=0
    
    vid=[]
    if abnor==3:
        prediction.append('abnormal')
    else:
        prediction.append('normal')

In [ ]:
len(prediction)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(orignal, prediction)
sns.heatmap(conf_mat, square=True, annot=True, cmap='Blues', fmt='d', cbar=False)